# 07. Test Inference 및 제출 파일 생성
- LSTM 모델을 이용해 `test_FD001.txt` 데이터에 대한 RUL 예측
- 예측 결과를 `.csv` 제출 파일 형태로 저장
- 실제 RUL (`RUL_FD001.txt`)과 비교

In [2]:
# 1. 필요 라이브러리 불러오기
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
# 2. 데이터 불러오기
test_df = pd.read_csv('../data/CMaps/test_FD001.txt', sep='\s+', header=None)
rul_df = pd.read_csv('../data/CMaps/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])
column_names = ['unit', 'time', 'op1', 'op2', 'op3'] + [f'sensor{i}' for i in range(1, 22)]
test_df.columns = column_names
features = ['op1', 'op2', 'sensor2', 'sensor3', 'sensor4', 'sensor7', 'sensor8',
            'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15',
            'sensor17', 'sensor20', 'sensor21']

In [4]:
# 3. 유닛별로 마지막 시점 윈도우 생성 함수
def create_test_windows(test_df, window_size=30):
    test_units = []
    for unit in test_df['unit'].unique():
        unit_df = test_df[test_df['unit'] == unit].copy().reset_index(drop=True)
        if len(unit_df) < window_size:
            pad_df = pd.DataFrame([unit_df.iloc[0]] * (window_size - len(unit_df)))
            unit_df = pd.concat([pad_df, unit_df], ignore_index=True)
        else:
            unit_df = unit_df.iloc[-window_size:]
        test_units.append(unit_df[features].values)
    return np.array(test_units)

# 정규화 (주의: 실제로는 train에서 사용한 scaler를 로드해야 함)
scaler = MinMaxScaler()
test_df[features] = scaler.fit_transform(test_df[features])  # 예시 목적
X_test = create_test_windows(test_df)
print("Test shape:", X_test.shape)

Test shape: (100, 30, 15)


In [5]:
# 4. 모델 로드 및 예측
model = load_model('../models/lstm_model.h5')
y_pred = model.predict(X_test).flatten()
y_true = rul_df['RUL'].values
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print(f"Test MAE: {mae:.2f}, RMSE: {rmse:.2f}")

4/4 [==============================] - 0s 1ms/step
Test MAE: 30.09, RMSE: 36.49


In [6]:
# 5. 제출 파일 저장
submission = pd.DataFrame({
    'unit': test_df['unit'].unique(),
    'RUL_pred': y_pred
})
submission.to_csv('../reports/submission_lstm.csv', index=False)
submission.head()

,unit,RUL_pred
0,1,103.148773
1,2,80.403236
2,3,18.952341
3,4,42.418972
4,5,48.784290


In [7]:
# LSTM 예측 결과 저장
submission.to_csv('../models/lstm_pred.csv', index=False)